In [1]:
from valor import (
    Dataset, 
    Model, 
    Datum,
    GroundTruth, 
    Prediction, 
    Annotation, 
    Label, 
    Client,
    connect,
)
from valor.schemas import Embedding
from valor.enums import EvaluationStatus, MetricType

connect("http://localhost:8000")
client = Client()

Successfully connected to host at http://localhost:8000/


In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModel

tiny_imagenet = load_dataset('zh-plus/tiny-imagenet', split='train')
processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
dinov2 = AutoModel.from_pretrained('facebook/dinov2-small')

/home/czaloom/valor/.env-velour/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def embed(x) -> Embedding:
    inputs = processor(images=x, return_tensors="pt")
    outputs = dinov2(**inputs)
    retval = (
        outputs
        .last_hidden_state
        .flatten()
        .detach()
        .numpy()
    )
    retval = retval.tolist()
    return Embedding(retval)

In [4]:
classes = {
    0: "fish",
    1: "ground_spider",
    2: "frog",
    5: "snake",
    8: "web_spider",
    19: "penguin"
}
label_ids = classes.keys()
text_labels = list(classes.values())

Create ground truths and predictions.

In [ ]:
dataset = Dataset.create("tiny-imagenet")
model = Model.create("facebook/dinov2-small")

In [6]:
groundtruths = []
predictions = []

for idx, image in enumerate(tiny_imagenet):
    label_id = image['label']
    pil_image = image['image']

    # ignore other labels
    if label_id in label_ids:
        datum = Datum(uid=str(idx))
        dataset.add_groundtruth(
            GroundTruth(
                datum=datum,
                annotations=[
                    Annotation(
                        labels=[
                            Label(
                                key="class", 
                                value=classes[label_id]
                            )
                        ]
                    )
                ]
            )
        )
        model.add_prediction(
            dataset,
            Prediction(
                datum=datum,
                annotations=[
                    Annotation(
                        embedding=embed(pil_image)
                    )
                ]
            )
        )

ClientException: {"name": "OperationalError", "detail": "(psycopg2.errors.ProgramLimitExceeded) vector cannot have more than 16000 dimensions\nLINE 1: INSERT INTO embedding (value, created_at) VALUES ('[-2.21651...\n                                                          ^\n\n[SQL: INSERT INTO embedding (value, created_at) VALUES (%(value)s, now()) RETURNING embedding.id, embedding.created_at]\n[parameters: {'value': '[-2.216519832611084,-0.4405917823314667,-0.09975497424602509,-0.6448603272438049,-0.19376243650913239,0.6705418825149536,1.8325440883636475,-1.867935 ... (1902918 characters truncated) ... 888239860535,-1.5801950693130493,2.1737005710601807,-1.536237359046936,-5.610318183898926,1.4012024402618408,-1.9985246658325195,-3.3968770503997803]'}]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)", "timestamp": 1719450608.231918}

In [ ]:

dataset.finalize()

Compute metrics

In [ ]:
job = model.evaluate_embeddings(dataset, limit=10)
assert job.wait_for_completion(timeout=30) == EvaluationStatus.DONE

cvm = job.metrics[0] if job.metrics[0]['type'] == MetricType.CramerVonMises else job.metrics[1]
ks = job.metrics[0] if job.metrics[0]['type'] == MetricType.KolmgorovSmirnov else job.metrics[1]

Create pandas dataframes

In [ ]:
def create_dataframe(x: dict, labels: list[str]):
    col_ix = pd.MultiIndex.from_product([["Reference"], labels])
    row_ix = pd.MultiIndex.from_product([["Query"], labels])

    statistic_df = pd.DataFrame(x["value"]["statistics"])
    statistic_df = statistic_df.set_index(row_ix)
    statistic_df.columns = col_ix

    pvalue_df = pd.DataFrame(x["value"]["pvalues"])
    pvalue_df = pvalue_df.set_index(row_ix)
    pvalue_df.columns = col_ix

    return (statistic_df, pvalue_df)

cvm_statistics, cvm_pvalues = create_dataframe(cvm, text_labels)
ks_statistics, ks_pvalues = create_dataframe(ks, text_labels)

In [ ]:
cvm_statistics

In [ ]:
cvm_pvalues

In [ ]:
ks_statistics

In [ ]:
ks_pvalues